# American Option Pricing using Least Square Monte-Carlo Simulation non-recursively. 
Simplified method derived from part 1 of the paper *Valuing American Options by Simulation: A Simple Least-Squares Approach* by Francis A. Longstaff and Eduardo S. Schwartz.

In [1]:
# imports
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# Stock price paths
t0 = [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00]
t1 = [1.09, 1.16, 1.22, 0.93, 1.11, 0.76, 0.92, 0.88]
t2 = [1.08, 1.26, 1.07, 0.97, 1.56, 0.77, 0.84, 1.22]
t3 = [1.34, 1.54, 1.03, 0.92, 1.52, 0.90, 1.01, 1.34]
stock_price_paths = pd.DataFrame({'t=0': t0, 't=1': t1, 't=2': t2, 't=3': t3})
stock_price_paths

,t=0,t=1,t=2,t=3
0,1.0,1.09,1.08,1.34
1,1.0,1.16,1.26,1.54
2,1.0,1.22,1.07,1.03
3,1.0,0.93,0.97,0.92
4,1.0,1.11,1.56,1.52
5,1.0,0.76,0.77,0.90
6,1.0,0.92,0.84,1.01
7,1.0,0.88,1.22,1.34


In [3]:
# Cash flow matrix at time 3
exercise_cf_t3 = []
for i in t3:
    if i >= 1.1:
        exercise_cf_t3.append(0.)
    else:
        exercise_cf_t3.append(1.1-i)
cashflows_t3 = pd.DataFrame({'t=0': np.zeros(8), 't=1': np.zeros(8), 't=2': np.zeros(8), 't=3': exercise_cf_t3})

cashflows_t3

,t=0,t=1,t=2,t=3
0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.0,0.00
2,0.0,0.0,0.0,0.07
3,0.0,0.0,0.0,0.18
4,0.0,0.0,0.0,0.00
5,0.0,0.0,0.0,0.20
6,0.0,0.0,0.0,0.09
7,0.0,0.0,0.0,0.00


In [4]:
# initialization
rf = 0.06 # risk free rate
X = []
Y = []

for i in range(len(t2)):
    if t2[i] < 1.1:
        X.append(t2[i])
        Y.append(exercise_cf_t3[i])

X = np.array(X).reshape(-1, 1)
Y = np.array(Y) * np.exp(-rf)
        
# Create a DataFrame
data = pd.DataFrame({'X': X.flatten(), 'Y': Y.flatten()})

# Create polynomial features
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Perform quadratic regression
model = LinearRegression()
model.fit(X_poly, Y)

# Get coefficients
reg_t2_intercept = model.intercept_
reg_t2_coef_x = model.coef_[1]
reg_t2_coef_x2 = model.coef_[2]

# Print the result
print(f"Regression Result:")
print(f"Y = {reg_t2_intercept:.3f} + {reg_t2_coef_x:.3f}x + {reg_t2_coef_x2:.3f}x^2")

reg_t2_df = pd.DataFrame({'Y': Y.flatten(), 'X': X.flatten()})
reg_t2_df

Regression Result:
Y = -1.070 + 2.983x + -1.814x^2


,Y,X
0,0.000000,1.08
1,0.065924,1.07
2,0.169518,0.97
3,0.188353,0.77
4,0.084759,0.84


In [5]:
# Optimal early exercise decision at time 2
exercise_cf_t2 = []
continuation_cf_t2 = []

for i in range(len(t2)):
    if t2[i] < 1.1:
        exercise_cf_t2.append(1.1-t2[i])
        continuation_cf_t2.append(reg_t2_intercept + reg_t2_coef_x * t2[i] + reg_t2_coef_x2 * t2[i]**2)
        
cashflows_early_vs_conti_t2 = pd.DataFrame({'Exercise': exercise_cf_t2, 'Continuation': continuation_cf_t2})
cashflows_early_vs_conti_t2

,Exercise,Continuation
0,0.02,0.036741
1,0.03,0.045898
2,0.13,0.117527
3,0.33,0.151969
4,0.26,0.156418


In [6]:
# Cash flow matrix at time 2
cashflows_t2 = cashflows_t3.copy()

for i in range(len(t2)):
    c = reg_t2_intercept + reg_t2_coef_x * t2[i] + reg_t2_coef_x2 * t2[i]**2 # continuation
    e = 1.1 - t2[i] # exercise
    if t2[i] < 1.1:
        if e >= c:
            cashflows_t2['t=3'][i] = 0
            cashflows_t2['t=2'][i] = e
        
# cashflows_t3 = pd.DataFrame({'t=0': np.zeros(8), 't=1': np.zeros(8), 't=2': np.zeros(8), 't=3': exercise_cf_t3})
cashflows_t2

,t=0,t=1,t=2,t=3
0,0.0,0.0,0.00,0.00
1,0.0,0.0,0.00,0.00
2,0.0,0.0,0.00,0.07
3,0.0,0.0,0.13,0.00
4,0.0,0.0,0.00,0.00
5,0.0,0.0,0.33,0.00
6,0.0,0.0,0.26,0.00
7,0.0,0.0,0.00,0.00


In [7]:
# Same process for time 1
# initialization
rf = 0.06 # risk free rate
X = []
Y = []

for i in range(len(t1)):
    if t1[i] < 1.1:
        X.append(t1[i])
        Y.append(cashflows_t2['t=2'][i])

X = np.array(X).reshape(-1, 1)
Y = np.array(Y) * np.exp(-rf)

# Create a DataFrame
data = pd.DataFrame({'X': X.flatten(), 'Y': Y.flatten()})

# Create polynomial features
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Perform quadratic regression
model = LinearRegression()
model.fit(X_poly, Y)

# Get coefficients
reg_t1_intercept = model.intercept_
reg_t1_coef_x = model.coef_[1]
reg_t1_coef_x2 = model.coef_[2]

# Print the result
print(f"Regression Result:")
print(f"Y = {reg_t1_intercept:.3f} + {reg_t1_coef_x:.3f}x + {reg_t1_coef_x2:.3f}x^2")

reg_t1_df = pd.DataFrame({'Y': Y.flatten(), 'X': X.flatten()})
reg_t1_df

Regression Result:
Y = 2.038 + -3.335x + 1.356x^2


,Y,X
0,0.000000,1.09
1,0.122429,0.93
2,0.310782,0.76
3,0.244859,0.92
4,0.000000,0.88


In [8]:
# Optimal early exercise decision at time 1
exercise_cf_t1 = []
continuation_cf_t1 = []

for i in range(len(t1)):
    if t1[i] < 1.1:
        exercise_cf_t1.append(1.1-t1[i])
        continuation_cf_t1.append(reg_t1_intercept + reg_t1_coef_x * t1[i] + reg_t1_coef_x2 * t1[i]**2)
        
cashflows_early_vs_conti_t1 = pd.DataFrame({'Exercise': exercise_cf_t1, 'Continuation': continuation_cf_t1})
cashflows_early_vs_conti_t1

,Exercise,Continuation
0,0.01,0.013485
1,0.17,0.108749
2,0.34,0.286065
3,0.18,0.117009
4,0.22,0.152762


In [9]:
# Cash flow matrix at time 1
cashflows_t1 = cashflows_t2.copy()

for i in range(len(t1)):
    c = reg_t1_intercept + reg_t1_coef_x * t1[i] + reg_t1_coef_x2 * t1[i]**2 # continuation
    e = 1.1 - t1[i] # exercise
    if t1[i] < 1.1:
        if e >= c:
            cashflows_t1['t=2'][i] = 0
            cashflows_t1['t=1'][i] = e
# cashflows_t1 = cashflows_t1.drop(columns='t=0')
cashflows_t1

,t=0,t=1,t=2,t=3
0,0.0,0.00,0.0,0.00
1,0.0,0.00,0.0,0.00
2,0.0,0.00,0.0,0.07
3,0.0,0.17,0.0,0.00
4,0.0,0.00,0.0,0.00
5,0.0,0.34,0.0,0.00
6,0.0,0.18,0.0,0.00
7,0.0,0.22,0.0,0.00


In [10]:
stopping_rule = cashflows_t1.applymap(lambda x: 1 if x != 0 else 0)
stopping_rule

,t=0,t=1,t=2,t=3
0,0,0,0,0
1,0,0,0,0
2,0,0,0,1
3,0,1,0,0
4,0,0,0,0
5,0,1,0,0
6,0,1,0,0
7,0,1,0,0


In [11]:
# Valuing the option at time 0
cashflows_t1 = cashflows_t1.drop(columns='t=0') #warning run this cell once or if rerun then comment this line to avoid errors.
time_period = 1
paths = 8
weighted_sum = 0
for col in cashflows_t1.columns:
    for i in cashflows_t1[col]:
        if i != 0:
            weighted_sum += i * np.exp(-rf * time_period)
            
        
    time_period += 1
print(weighted_sum/paths)

0.11443433004505696
